In [4]:
"""
Movie Recommendation System - Jupyter Notebook Version
Works without ipywidgets - Pure Python with HTML Display
"""

# Import libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML

# -----------------------------
# CUSTOM STYLING
# -----------------------------
style = """
<style>
.movie-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    padding: 30px;
    border-radius: 15px;
    text-align: center;
    color: white;
    margin-bottom: 30px;
}
.movie-card {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    padding: 20px;
    border-radius: 10px;
    margin: 15px 0;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    color: white;
}
.movie-number {
    font-size: 2rem;
    font-weight: bold;
    opacity: 0.8;
    display: inline-block;
    width: 60px;
}
.movie-title {
    font-size: 1.3rem;
    font-weight: 600;
    display: inline-block;
}
.stats-box {
    background: #f8f9fa;
    padding: 20px;
    border-radius: 10px;
    text-align: center;
    border-left: 5px solid #667eea;
    margin: 10px;
    display: inline-block;
    min-width: 200px;
}
.success-msg {
    background: #d4edda;
    border: 1px solid #c3e6cb;
    color: #155724;
    padding: 15px;
    border-radius: 8px;
    margin: 15px 0;
}
.error-msg {
    background: #f8d7da;
    border: 1px solid #f5c6cb;
    color: #721c24;
    padding: 15px;
    border-radius: 8px;
    margin: 15px 0;
}
.info-box {
    background: #d1ecf1;
    border: 1px solid #bee5eb;
    color: #0c5460;
    padding: 15px;
    border-radius: 8px;
    margin: 15px 0;
}
.code-box {
    background: #f4f4f4;
    border-left: 4px solid #667eea;
    padding: 15px;
    border-radius: 5px;
    font-family: 'Courier New', monospace;
    margin: 10px 0;
}
</style>
"""

display(HTML(style))

# Display header
header_html = """
<div class="movie-header">
    <h1 style="margin: 0; font-size: 3rem;">🎬 Movie Recommendation System</h1>
    <p style="margin-top: 10px; font-size: 1.2rem; opacity: 0.9;">
        Discover your next favorite movie based on what you love
    </p>
</div>
"""
display(HTML(header_html))

# -----------------------------
# LOAD DATA
# -----------------------------
print("📊 Loading movie data...")
movies = pd.read_csv("tmdb_5000_movies.csv")
movies = movies[['title', 'overview']]
movies['overview'] = movies['overview'].fillna('')

# Display stats
stats_html = f"""
<div style="text-align: center;">
    <div class="stats-box">
        <h2 style="margin: 0; color: #667eea;">{len(movies):,}</h2>
        <p style="margin: 5px 0 0 0; color: #666;">Total Movies</p>
    </div>
    <div class="stats-box">
        <h2 style="margin: 0; color: #667eea;">5</h2>
        <p style="margin: 5px 0 0 0; color: #666;">Recommendations Per Query</p>
    </div>
</div>
"""
display(HTML(stats_html))

# -----------------------------
# COMPUTE SIMILARITY
# -----------------------------
print("\n🔄 Computing movie similarities (this may take a moment)...")
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['overview'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(movies.index, index=movies['title'].str.lower()).drop_duplicates()

print("✅ System ready!\n")

# -----------------------------
# RECOMMEND FUNCTION
# -----------------------------
def recommend_movies(movie_title):
    """
    Get movie recommendations based on a movie title.
    
    Parameters:
    -----------
    movie_title : str
        The title of the movie
    
    Returns:
    --------
    None (displays HTML output directly)
    """
    movie_title_lower = movie_title.strip().lower()
    
    if not movie_title_lower:
        display(HTML('<div class="error-msg">⚠️ <strong>Please enter a movie name!</strong></div>'))
        return
    
    if movie_title_lower not in indices:
        error_html = f'<div class="error-msg">❌ <strong>Movie "{movie_title}" not found.</strong> Please check the spelling or try another movie.</div>'
        display(HTML(error_html))
        
        # Show suggestions
        if len(movie_title_lower) >= 3:
            similar = movies[movies['title'].str.lower().str.contains(movie_title_lower[:3], na=False)]['title'].head(5)
            
            if not similar.empty:
                suggestions_html = '<div class="info-box"><strong>💡 Did you mean:</strong><ul style="margin: 10px 0;">'
                for movie in similar:
                    suggestions_html += f'<li>{movie}</li>'
                suggestions_html += '</ul></div>'
                display(HTML(suggestions_html))
        return
    
    idx = indices[movie_title_lower]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    movie_indices = [i[0] for i in sim_scores]
    
    recommendations = movies['title'].iloc[movie_indices]
    original_title = movies['title'].iloc[idx]
    
    success_html = f'<div class="success-msg">✅ <strong>Based on "{original_title}", here are your recommendations:</strong></div>'
    display(HTML(success_html))
    
    # Display recommendations
    for i, movie in enumerate(recommendations, 1):
        card_html = f"""
        <div class="movie-card">
            <span class="movie-number">#{i}</span>
            <span class="movie-title">{movie}</span>
        </div>
        """
        display(HTML(card_html))

# -----------------------------
# USAGE INSTRUCTIONS
# -----------------------------
info_html = """
<div class="info-box">
    <h3 style="margin-top: 0;">💡 How to use:</h3>
    <ol style="margin: 10px 0;">
        <li>Run the cell below with your movie name</li>
        <li>Or create a new cell and call: <code>recommend_movies('Your Movie Name')</code></li>
        <li>Get 5 similar movies based on plot descriptions</li>
    </ol>
    <h4>🎯 Try these popular movies:</h4>
    <p>The Dark Knight • Avatar • Inception • Interstellar • The Matrix • Titanic • Pulp Fiction</p>
</div>
"""
display(HTML(info_html))

# -----------------------------
# EXAMPLE USAGE
# -----------------------------
example_html = """
<div class="code-box">
    <strong>📝 Example Usage:</strong><br><br>
    <code>recommend_movies('The Dark Knight')</code><br>
    <code>recommend_movies('Avatar')</code><br>
    <code>recommend_movies('Inception')</code>
</div>
"""
display(HTML(example_html))

print("\n" + "="*80)
print("🎬 READY TO USE! Run the cell below or create a new cell with:")
print("   recommend_movies('Your Movie Name')")
print("="*80)

📊 Loading movie data...



🔄 Computing movie similarities (this may take a moment)...
✅ System ready!




🎬 READY TO USE! Run the cell below or create a new cell with:
   recommend_movies('Your Movie Name')


In [6]:
recommend_movies('The Dark Knight')